# AI Config Update - Cookbook

This cookbook contains all code from `aiconfig-update/SKILL.md` for updating, archiving, and deleting AI Configs.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `ai-configs:write` permission
- AI Config created (run `cookbook_aiconfig_create.ipynb` first)

In [1]:
# Install and load environment
%pip install requests python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


---
## Update AI Config
From: `SKILL.md` lines 31-63

In [2]:
import requests
import json
import os

# Configuration - use same project as aiconfig-create cookbook
API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"
CONFIG_KEY = "content-assistant"  # REAL config to test

# Update config properties
url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{CONFIG_KEY}"

headers = {
    "Authorization": f"{API_TOKEN}",
    "Content-Type": "application/json"
}

# Update data - simple object with fields to update
patch_data = {
    "description": "Content assistant AI Config - updated via cookbook test"
}

response = requests.patch(url, headers=headers, json=patch_data)

if response.status_code == 200:
    print("[OK] AI Config updated successfully")
    data = response.json()
    print(f"  Name: {data.get('name')}")
    print(f"  Description: {data.get('description')}")
else:
    print(f"[ERROR] {response.status_code}: {response.text}")

[OK] AI Config updated successfully
  Name: Content Generation Assistant
  Description: Content assistant AI Config - updated via cookbook test


---
## Archive AI Config
From: `SKILL.md` lines 69-96

In [3]:
def archive_config(project_key, config_key, api_token, archive=True):
    """Archive or unarchive an AI Config"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs/{config_key}"

    headers = {
        "Authorization": f"{api_token}",
        "Content-Type": "application/json"
    }

    patch_data = {"archived": archive}

    response = requests.patch(url, headers=headers, json=patch_data)

    if response.status_code == 200:
        status = "archived" if archive else "unarchived"
        print(f"[OK] AI Config {status} successfully")
        return response.json()
    else:
        print(f"[ERROR] {response.status_code}: {response.text}")
        return None

print("[OK] archive_config() defined")

[OK] archive_config() defined


In [4]:
# Test archive_config with REAL config
# Note: We won't actually archive the main config - just verify the function works
# Test by archiving then immediately unarchiving
print("=== Testing archive_config (archive -> unarchive cycle) ===")

# Archive
result = archive_config(PROJECT_KEY, CONFIG_KEY, API_TOKEN, archive=True)
if result:
    print(f"  Archived: {result.get('archived')}")
    
    # Immediately unarchive to restore
    result = archive_config(PROJECT_KEY, CONFIG_KEY, API_TOKEN, archive=False)
    if result:
        print(f"  Unarchived: {not result.get('archived')}")
        print("[OK] archive_config() works with real config")
    else:
        print("[WARNING] Failed to unarchive")
else:
    print("[INFO] Archive may have failed - check API permissions")

=== Testing archive_config (archive -> unarchive cycle) ===
[OK] AI Config archived successfully
  Archived: None
[OK] AI Config unarchived successfully
  Unarchived: True
[OK] archive_config() works with real config


---
## Delete AI Config
From: `SKILL.md` lines 102-131

In [5]:
def delete_config(project_key, config_key, api_token, confirm=False):
    """Permanently delete an AI Config

    Args:
        confirm: Must be True to actually delete (safety check)
    """
    url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs/{config_key}"

    headers = {
        "Authorization": f"{api_token}"
    }

    if not confirm:
        print(f"[WARNING] Delete skipped - set confirm=True to delete '{config_key}'")
        return False

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"[OK] AI Config '{config_key}' deleted successfully")
        return True
    else:
        print(f"[ERROR] deleting config: {response.status_code}")
        print(response.text)
        return False

print("[OK] delete_config() defined")

# Test delete by creating a temporary config then deleting it
print("=== Testing delete_config with temporary config ===")

# 1. Create a temporary config for testing delete
TEMP_CONFIG_KEY = "cookbook-delete-test"
create_url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs"
create_headers = {
    "Authorization": f"{API_TOKEN}",
    "Content-Type": "application/json"
}
create_data = {
    "key": TEMP_CONFIG_KEY,
    "name": "Cookbook Delete Test",
    "description": "Temporary config for testing delete",
    "mode": "completion",
    "variations": [
        {
            "name": "Default",
            "model": {"name": "claude-3-5-haiku-latest"},
            "messages": [{"role": "system", "content": "Test"}]
        }
    ]
}

response = requests.post(create_url, headers=create_headers, json=create_data)
if response.status_code in [200, 201]:
    print(f"[OK] Created temporary config '{TEMP_CONFIG_KEY}'")
    
    # 2. Test the safety check (confirm=False)
    result = delete_config(PROJECT_KEY, TEMP_CONFIG_KEY, API_TOKEN, confirm=False)
    if result == False:
        print("[OK] Safety check works (confirm=False prevents delete)")
    
    # 3. Actually delete the config
    result = delete_config(PROJECT_KEY, TEMP_CONFIG_KEY, API_TOKEN, confirm=True)
    if result:
        print("[OK] delete_config() works with real deletion")
    else:
        print("[ERROR] Delete failed")
elif response.status_code == 409:
    # Config already exists from previous run - just delete it
    print(f"[INFO] Config '{TEMP_CONFIG_KEY}' already exists, deleting it")
    result = delete_config(PROJECT_KEY, TEMP_CONFIG_KEY, API_TOKEN, confirm=True)
    print("[OK] Cleaned up existing test config")
else:
    print(f"[ERROR] Failed to create temp config: {response.status_code}")
    print(response.text)

[OK] delete_config() defined
=== Testing delete_config with temporary config ===
[OK] Created temporary config 'cookbook-delete-test'
[WARNING] Delete skipped - set confirm=True to delete 'cookbook-delete-test'
[OK] Safety check works (confirm=False prevents delete)
[OK] AI Config 'cookbook-delete-test' deleted successfully
[OK] delete_config() works with real deletion


---
## Batch Operations
From: `SKILL.md` lines 137-170

In [6]:
def batch_update_configs(project_key, updates, api_token):
    """Update multiple AI Configs"""
    results = []

    for config_key, patch_data in updates.items():
        url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs/{config_key}"

        headers = {
            "Authorization": f"{api_token}",
            "Content-Type": "application/json"
        }

        response = requests.patch(url, headers=headers, json=patch_data)

        results.append({
            "config_key": config_key,
            "success": response.status_code == 200,
            "status_code": response.status_code
        })

    return results

print("[OK] batch_update_configs() defined")

# Test batch update with REAL configs
print("=== Testing batch_update_configs ===")
updates = {
    "content-assistant": {"description": "Batch updated via cookbook"},
    "support-agent": {"description": "Batch updated via cookbook"}
}

results = batch_update_configs(PROJECT_KEY, updates, API_TOKEN)
for result in results:
    status = "[OK]" if result['success'] else "[ERROR]"
    print(f"  {status} {result['config_key']}: {result['status_code']}")

[OK] batch_update_configs() defined
=== Testing batch_update_configs ===
  [OK] content-assistant: 200
  [OK] support-agent: 200


---
## Error Handling
From: `SKILL.md` lines 187-226

In [7]:
def safe_update_config(project_key, config_key, patch_data, api_token):
    """Update config with comprehensive error handling"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs/{config_key}"

    headers = {
        "Authorization": f"{api_token}",
        "Content-Type": "application/json"
    }

    try:
        response = requests.patch(url, headers=headers, json=patch_data)

        if response.status_code == 200:
            return {"success": True, "data": response.json()}
        elif response.status_code == 400:
            return {"success": False, "error": "Invalid request format"}
        elif response.status_code == 404:
            return {"success": False, "error": "AI Config not found"}
        elif response.status_code == 409:
            return {"success": False, "error": "Conflict - config may have been modified"}
        else:
            return {"success": False, "error": f"Unexpected error: {response.status_code}"}

    except requests.exceptions.RequestException as e:
        return {"success": False, "error": f"Request failed: {str(e)}"}

print("[OK] safe_update_config() defined")

# Test safe_update_config with REAL config
print("=== Testing safe_update_config ===")
result = safe_update_config(
    PROJECT_KEY,
    CONFIG_KEY,
    {"description": "Safe update test"},
    API_TOKEN
)

if result["success"]:
    print(f"[OK] Update successful")
    print(f"  Name: {result['data'].get('name')}")
else:
    print(f"[ERROR] Update failed: {result['error']}")

# Test error handling with non-existent config
print("\n=== Testing error handling (404 case) ===")
result = safe_update_config(
    PROJECT_KEY,
    "non-existent-config-12345",
    {"description": "Test"},
    API_TOKEN
)
if not result["success"] and "not found" in result["error"].lower():
    print(f"[OK] Error handling works: {result['error']}")
else:
    print(f"[INFO] Got: {result}")

[OK] safe_update_config() defined
=== Testing safe_update_config ===
[OK] Update successful
  Name: Content Generation Assistant

=== Testing error handling (404 case) ===
[OK] Error handling works: AI Config not found


---
## Complete AIConfigManager Class
From: `SKILL.md` lines 247-284

In [8]:
class AIConfigManager:
    def __init__(self, project_key, api_token):
        self.project_key = project_key
        self.api_token = api_token
        self.base_url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}"
        self.headers = {
            "Authorization": api_token,
            "Content-Type": "application/json"
        }

    def update(self, config_key, patch_data):
        """Update an AI Config"""
        url = f"{self.base_url}/ai-configs/{config_key}"
        response = requests.patch(url, headers=self.headers, json=patch_data)
        return response.status_code == 200, response

    def archive(self, config_key):
        """Archive an AI Config"""
        return self.update(config_key, {"archived": True})

    def unarchive(self, config_key):
        """Unarchive an AI Config"""
        return self.update(config_key, {"archived": False})

    def delete(self, config_key):
        """Delete an AI Config"""
        url = f"{self.base_url}/ai-configs/{config_key}"
        response = requests.delete(url, headers={"Authorization": self.api_token})
        return response.status_code == 204

print("[OK] AIConfigManager class defined")

# Test AIConfigManager with REAL config
print("=== Testing AIConfigManager ===")
manager = AIConfigManager(PROJECT_KEY, API_TOKEN)

# Test update
success, response = manager.update(CONFIG_KEY, {"description": "Updated via AIConfigManager"})
print(f"  Update: {'[OK]' if success else '[ERROR]'}")

# Test archive/unarchive cycle (quick test)
print("  Testing archive -> unarchive cycle...")
success, _ = manager.archive(CONFIG_KEY)
print(f"    Archive: {'[OK]' if success else '[ERROR]'}")

success, _ = manager.unarchive(CONFIG_KEY)
print(f"    Unarchive: {'[OK]' if success else '[ERROR]'}")

print("[OK] AIConfigManager works with real config")

[OK] AIConfigManager class defined
=== Testing AIConfigManager ===
  Update: [OK]
  Testing archive -> unarchive cycle...
    Archive: [OK]
    Unarchive: [OK]
[OK] AIConfigManager works with real config
